In [1]:
# =============================================================================
# IMPORTAR LIBRERÍAS
# =============================================================================
print("\n" + "="*80)
print("IMPORTANDO LIBRERÍAS...")
print("="*80)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Para leer archivos SPSS
try:
    import pyreadstat
    print("✓ pyreadstat importado correctamente")
except ImportError:
    print("❌ ERROR: pyreadstat no está instalado")
    print("   Instalar con: pip install pyreadstat")
    exit()

# Preprocesamiento
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Modelos de clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

# Métricas de evaluación
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✓ Todas las librerías importadas correctamente")


IMPORTANDO LIBRERÍAS...
✓ pyreadstat importado correctamente
✓ Todas las librerías importadas correctamente


In [1]:
pip install pyreadstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.1 MB/s eta 0:00:00


In [2]:
# =============================================================================
# PASO 1: CARGAR DATOS DESDE SPSS
# =============================================================================
print("\n" + "="*80)
print("PASO 1: CARGANDO DATOS DESDE ARCHIVO SPSS")
print("="*80)

try:
    # Cargar archivo .sav de SPSS
    df, meta = pyreadstat.read_sav('CSALUD08_2024.sav')
    print(f"✓ Datos cargados exitosamente")
    print(f"  - Dimensiones: {df.shape[0]:,} filas x {df.shape[1]} columnas")
except FileNotFoundError:
    print("❌ ERROR: No se encontró el archivo 'CSALUD08_2024.sav'")
    print("   Asegúrate de que el archivo esté en la misma carpeta que este script")
    exit()
except Exception as e:
    print(f"❌ ERROR al cargar datos: {e}")
    exit()

print(f"\n✓ Primeras 5 filas del dataset:")
print(df.head())

print(f"\n✓ Columnas disponibles ({len(df.columns)}):")
print(list(df.columns))


PASO 1: CARGANDO DATOS DESDE ARCHIVO SPSS
✓ Datos cargados exitosamente
  - Dimensiones: 41,309 filas x 53 columnas

✓ Primeras 5 filas del dataset:
      ID1             HHID  QHCLUSTER  QHNUMBER  QHHOME  QS800  QS801  QS802  \
0  2024.0        325503101     3255.0      31.0     1.0    NaN    3.0    2.0   
1  2024.0        325504701     3255.0      47.0     1.0    NaN    3.0    2.0   
2  2024.0        325504701     3255.0      47.0     1.0    NaN    4.0    2.0   
3  2024.0        325505001     3255.0      50.0     1.0    NaN    3.0    1.0   
4  2024.0        325505301     3255.0      53.0     1.0    NaN    2.0    1.0   

   QS802V  QS802A  ...  QS832  QS833  QS834  QS835  QS836  QS837  QS838  \
0     1.0     1.0  ...    NaN    NaN    NaN    NaN    NaN    NaN          
1     1.0     1.0  ...    2.0    1.0    1.0    1.0    2.0    2.0      C   
2     1.0     1.0  ...    NaN    NaN    NaN    NaN    NaN    NaN          
3     1.0     1.0  ...    NaN    NaN    NaN    NaN    NaN    NaN     

In [3]:
# =============================================================================
# PASO 2: LIMPIEZA DE DATOS
# =============================================================================
print("\n" + "="*80)
print("PASO 2: LIMPIEZA DE DATOS")
print("="*80)

# 2.1 Valores perdidos iniciales
print("\n2.1 Análisis de valores perdidos:")
missing_before = df.isnull().sum().sum()
print(f"  Total de valores perdidos: {missing_before:,}")

# 2.2 Crear copia del dataframe
df_clean = df.copy()

# 2.3 Filtrar variable objetivo QS810
print("\n2.2 Filtrando variable objetivo (QS810)...")
print(f"  Registros antes: {len(df_clean):,}")
print(f"  Distribución original:")
print(df_clean['QS810'].value_counts(dropna=False))

# Mantener solo respuestas válidas (1=Sí, 2=No)
df_clean = df_clean[df_clean['QS810'].isin([1, 2])].copy()
print(f"  Registros después: {len(df_clean):,}")
print(f"  Registros eliminados: {len(df) - len(df_clean):,}")

# Recodificar variable objetivo (1=Sí->1, 2=No->0)
df_clean['QS810'] = df_clean['QS810'].map({1: 1, 2: 0})

print(f"\n  Variable objetivo recodificada:")
print(df_clean['QS810'].value_counts())
print(f"  Proporción:")
print(df_clean['QS810'].value_counts(normalize=True).round(3))



PASO 2: LIMPIEZA DE DATOS

2.1 Análisis de valores perdidos:
  Total de valores perdidos: 891,450

2.2 Filtrando variable objetivo (QS810)...
  Registros antes: 41,309
  Distribución original:
QS810
1.0    25019
NaN    10155
2.0     6119
8.0       16
Name: count, dtype: int64
  Registros después: 31,138
  Registros eliminados: 10,171

  Variable objetivo recodificada:
QS810
1    25019
0     6119
Name: count, dtype: int64
  Proporción:
QS810
1    0.803
0    0.197
Name: proportion, dtype: float64


In [4]:
# =============================================================================
# PASO 2.5: MANEJO DE VARIABLES CATEGÓRICAS
# =============================================================================
print("\n" + "="*80)
print("PASO 2.5: PROCESAMIENTO DE VARIABLES CATEGÓRICAS")
print("="*80)

# 2.5.1 Variables binarias (1=Sí, 2=No) -> (1, 0)
print("\n2.5.1 Recodificando variables binarias...")

variables_binarias = {
    'QS803': 'Atendido por odontólogo',
    'QS806': 'Recibió información higiene bucal',
    'QS809': 'Usa cepillo dental',
    'QS814': 'Usa crema dental',
    'QS817': 'Evaluaron vista',
    'QS820': 'Problema de visión',
    'QS828': 'Ve TV/PC muy cerca',
    'QS833': 'Asistió escuela',
    'QS836': 'Confianza con profesor'
}

for var, desc in variables_binarias.items():
    if var in df_clean.columns:
        # Reemplazar códigos especiales con NaN
        df_clean.loc[df_clean[var].isin([8, 9]), var] = np.nan

        # Recodificar (1->1, 2->0)
        if df_clean[var].dropna().isin([1, 2]).any():
            df_clean[var] = df_clean[var].map({1: 1, 2: 0})
            print(f"  ✓ {var}: {desc}")

# 2.5.2 Variables ordinales
print("\n2.5.2 Procesando variables ordinales...")

# QS811: Frecuencia de cepillado (1-4)
if 'QS811' in df_clean.columns:
    print(f"  ✓ QS811: Frecuencia cepillado (valores: {sorted(df_clean['QS811'].dropna().unique())})")

# QS834 y QS835: Frecuencia de golpes (1=Nunca, 2=Rara vez, 3=A veces, 4=A menudo)
for var in ['QS834', 'QS835']:
    if var in df_clean.columns:
        df_clean.loc[df_clean[var] == 8, var] = np.nan
        print(f"  ✓ {var}: Limpiado")

# 2.5.3 Variables categóricas -> Crear variables derivadas
print("\n2.5.3 Creando variables derivadas...")

# QS840B: Concentración de flúor -> Variable binaria
if 'QS840B' in df_clean.columns:
    # 1=1000ppm+, 2=601-999ppm, 3=<=600ppm, 4-9=Otros
    df_clean['fluor_adecuado'] = df_clean['QS840B'].apply(
        lambda x: 1 if x == 1 else (0 if x in [2, 3] else np.nan)
    )
    print(f"  ✓ Creada 'fluor_adecuado': {df_clean['fluor_adecuado'].value_counts().to_dict()}")

# QS807: Fuente de información -> Variables dummy
if 'QS807' in df_clean.columns:
    df_clean['info_minsa'] = df_clean['QS807'].astype(str).str.contains('A', na=False).astype(int)
    df_clean['info_escuela'] = df_clean['QS807'].astype(str).str.contains('I', na=False).astype(int)
    df_clean['info_familia'] = df_clean['QS807'].astype(str).str.contains('J', na=False).astype(int)
    df_clean['info_medios'] = df_clean['QS807'].astype(str).str.contains('H', na=False).astype(int)
    print(f"  ✓ Creadas variables dummy de fuentes de información")

# Quintil socioeconómico (si existe Pesomen12)
if 'Pesomen12' in df_clean.columns:
    try:
        df_clean['quintil_peso'] = pd.qcut(
            df_clean['Pesomen12'],
            q=5,
            labels=[1, 2, 3, 4, 5],
            duplicates='drop'
        ).astype(float)
        print(f"  ✓ Creada 'quintil_peso' (nivel socioeconómico)")
    except:
        print(f"  ⚠ No se pudo crear quintil_peso")

print(f"\n✓ Variables procesadas. Total de columnas: {df_clean.shape[1]}")



PASO 2.5: PROCESAMIENTO DE VARIABLES CATEGÓRICAS

2.5.1 Recodificando variables binarias...
  ✓ QS803: Atendido por odontólogo
  ✓ QS806: Recibió información higiene bucal
  ✓ QS809: Usa cepillo dental
  ✓ QS814: Usa crema dental
  ✓ QS817: Evaluaron vista
  ✓ QS820: Problema de visión
  ✓ QS828: Ve TV/PC muy cerca
  ✓ QS833: Asistió escuela
  ✓ QS836: Confianza con profesor

2.5.2 Procesando variables ordinales...
  ✓ QS811: Frecuencia cepillado (valores: [np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0)])
  ✓ QS834: Limpiado
  ✓ QS835: Limpiado

2.5.3 Creando variables derivadas...
  ✓ Creada 'fluor_adecuado': {1.0: 18674, 0.0: 2058}
  ✓ Creadas variables dummy de fuentes de información
  ✓ Creada 'quintil_peso' (nivel socioeconómico)

✓ Variables procesadas. Total de columnas: 59


In [5]:
# =============================================================================
# PASO 3: SELECCIÓN DE VARIABLES
# =============================================================================
print("\n" + "="*80)
print("PASO 3: SELECCIÓN DE VARIABLES PREDICTORAS")
print("="*80)

# Lista de variables predictoras
variables_predictoras = [
    # Demográficas
    'QS802D',           # Edad del niño

    # Salud bucal
    'QS803',            # Atendido por odontólogo
    'QS806',            # Recibió información
    'QS809',            # Usa cepillo
    'QS811',            # Frecuencia cepillado
    'QS814',            # Usa crema dental
    'fluor_adecuado',   # Flúor adecuado

    # Información
    'info_minsa',
    'info_escuela',
    'info_familia',
    'info_medios',

    # Salud ocular/mental
    'QS828',            # Ve TV muy cerca
    'QS834',            # Golpeado por profesor
    'QS835',            # Golpeado por estudiante
    'QS836',            # Confianza profesor

    # Socioeconómico
    'Pesomen12',        # Factor ponderación
    'quintil_peso'      # Quintil socioeconómico
]

# Filtrar variables disponibles
variables_disponibles = [v for v in variables_predictoras if v in df_clean.columns]
variables_faltantes = [v for v in variables_predictoras if v not in df_clean.columns]

print(f"\n✓ Variables disponibles: {len(variables_disponibles)}")
for i, var in enumerate(variables_disponibles, 1):
    print(f"  {i:2d}. {var}")

if variables_faltantes:
    print(f"\n⚠ Variables no disponibles: {len(variables_faltantes)}")
    for var in variables_faltantes:
        print(f"  - {var}")

# Crear dataset final
columnas_finales = variables_disponibles + ['QS810']
df_final = df_clean[columnas_finales].copy()

print(f"\n✓ Dataset final: {df_final.shape}")


PASO 3: SELECCIÓN DE VARIABLES PREDICTORAS

✓ Variables disponibles: 17
   1. QS802D
   2. QS803
   3. QS806
   4. QS809
   5. QS811
   6. QS814
   7. fluor_adecuado
   8. info_minsa
   9. info_escuela
  10. info_familia
  11. info_medios
  12. QS828
  13. QS834
  14. QS835
  15. QS836
  16. Pesomen12
  17. quintil_peso

✓ Dataset final: (31138, 18)


In [6]:
# =============================================================================
# PASO 3.5: IMPUTACIÓN DE VALORES PERDIDOS
# =============================================================================
print("\n" + "="*80)
print("PASO 3.5: IMPUTACIÓN DE VALORES PERDIDOS")
print("="*80)

print("\nValores perdidos por variable:")
missing_counts = df_final.isnull().sum()
missing_counts = missing_counts[missing_counts > 0].sort_values(ascending=False)
if len(missing_counts) > 0:
    for var, count in missing_counts.items():
        pct = (count / len(df_final)) * 100
        print(f"  {var:<20} {count:>6} ({pct:>5.1f}%)")

    # Imputar con mediana para variables numéricas
    print("\nImputando valores perdidos con mediana...")
    imputer = SimpleImputer(strategy='median')

    numeric_cols = [col for col in variables_disponibles if col in df_final.columns]
    df_final[numeric_cols] = imputer.fit_transform(df_final[numeric_cols])

    print(f"✓ Valores imputados")
else:
    print("✓ No hay valores perdidos")

print(f"\n✓ Valores perdidos finales: {df_final.isnull().sum().sum()}")


PASO 3.5: IMPUTACIÓN DE VALORES PERDIDOS

Valores perdidos por variable:
  QS836                 19167 ( 61.6%)
  QS834                 18805 ( 60.4%)
  QS835                 18803 ( 60.4%)
  fluor_adecuado        10406 ( 33.4%)
  QS806                 10212 ( 32.8%)
  QS828                  7011 ( 22.5%)
  QS811                  6119 ( 19.7%)

Imputando valores perdidos con mediana...
✓ Valores imputados

✓ Valores perdidos finales: 0


In [7]:
# =============================================================================
# PASO 4: SEPARAR X E Y
# =============================================================================
print("\n" + "="*80)
print("PASO 4: SEPARACIÓN DE CARACTERÍSTICAS Y OBJETIVO")
print("="*80)

X = df_final.drop('QS810', axis=1)
y = df_final['QS810']

print(f"\n✓ Características (X): {X.shape}")
print(f"  Columnas: {list(X.columns)}")

print(f"\n✓ Variable objetivo (y): {y.shape}")
print(f"  Distribución de clases:")
print(y.value_counts())
print(f"\n  Proporción:")
print(y.value_counts(normalize=True).round(3))

# Verificar balance
balance = y.value_counts(normalize=True).min()
if balance < 0.3:
    print("\n⚠ ADVERTENCIA: Clases desbalanceadas")
else:
    print("\n✓ Clases balanceadas")


PASO 4: SEPARACIÓN DE CARACTERÍSTICAS Y OBJETIVO

✓ Características (X): (31138, 17)
  Columnas: ['QS802D', 'QS803', 'QS806', 'QS809', 'QS811', 'QS814', 'fluor_adecuado', 'info_minsa', 'info_escuela', 'info_familia', 'info_medios', 'QS828', 'QS834', 'QS835', 'QS836', 'Pesomen12', 'quintil_peso']

✓ Variable objetivo (y): (31138,)
  Distribución de clases:
QS810
1    25019
0     6119
Name: count, dtype: int64

  Proporción:
QS810
1    0.803
0    0.197
Name: proportion, dtype: float64

⚠ ADVERTENCIA: Clases desbalanceadas


In [8]:
# =============================================================================
# PASO 5: DIVISIÓN TRAIN/TEST
# =============================================================================
print("\n" + "="*80)
print("PASO 5: DIVISIÓN TRAIN/TEST (80/20)")
print("="*80)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"\n✓ División completada:")
print(f"  Entrenamiento: {X_train.shape[0]:,} muestras ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  Prueba:        {X_test.shape[0]:,} muestras ({X_test.shape[0]/len(X)*100:.1f}%)")

print(f"\n✓ Distribución en entrenamiento:")
print(y_train.value_counts(normalize=True).round(3))

print(f"\n✓ Distribución en prueba:")
print(y_test.value_counts(normalize=True).round(3))


PASO 5: DIVISIÓN TRAIN/TEST (80/20)

✓ División completada:
  Entrenamiento: 24,910 muestras (80.0%)
  Prueba:        6,228 muestras (20.0%)

✓ Distribución en entrenamiento:
QS810
1    0.803
0    0.197
Name: proportion, dtype: float64

✓ Distribución en prueba:
QS810
1    0.803
0    0.197
Name: proportion, dtype: float64


In [9]:
# =============================================================================
# PASO 6: ESCALAMIENTO/NORMALIZACIÓN
# =============================================================================
print("\n" + "="*80)
print("PASO 6: ESCALAMIENTO CON STANDARDSCALER")
print("="*80)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convertir de vuelta a DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

print(f"\n✓ Datos escalados")
print(f"\nEstadísticas después del escalamiento (primeras 5 variables):")
print(X_train_scaled.describe().loc[['mean', 'std']].iloc[:, :5])



PASO 6: ESCALAMIENTO CON STANDARDSCALER

✓ Datos escalados

Estadísticas después del escalamiento (primeras 5 variables):
            QS802D         QS803         QS806  QS809         QS811
mean -1.537465e-16  1.257926e-16 -1.768513e-17    0.0  7.648104e-17
std   1.000020e+00  1.000020e+00  1.000020e+00    0.0  1.000020e+00


In [11]:
# =============================================================================
# FUNCIÓN AUXILIAR: EVALUAR MODELO CON GRÁFICOS
# =============================================================================
def evaluar_modelo(nombre_modelo, modelo, X_train, X_test, y_train, y_test, cv=5, numero_modelo=1):
    """Evalúa un modelo de clasificación de manera exhaustiva con visualizaciones"""

    print(f"\n{'='*80}")
    print(f"EVALUANDO: {nombre_modelo}")
    print('='*80)

    # Predicciones
    y_pred = modelo.predict(X_test)

    # Probabilidades
    try:
        y_pred_proba = modelo.predict_proba(X_test)[:, 1]
        tiene_proba = True
    except:
        y_pred_proba = None
        tiene_proba = False

    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    if tiene_proba:
        roc_auc = roc_auc_score(y_test, y_pred_proba)
    else:
        roc_auc = None

    # Validación cruzada
    cv_scores = cross_val_score(
        modelo, X_train, y_train,
        cv=StratifiedKFold(n_splits=cv, shuffle=True, random_state=42),
        scoring='accuracy'
    )

    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)

    # Mostrar resultados
    print(f"\n{'─'*80}")
    print("MÉTRICAS EN CONJUNTO DE PRUEBA:")
    print('─'*80)
    print(f"  Accuracy:     {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"  Precision:    {precision:.4f} ({precision*100:.2f}%)")
    print(f"  Recall:       {recall:.4f} ({recall*100:.2f}%)")
    print(f"  F1-Score:     {f1:.4f} ({f1*100:.2f}%)")
    if roc_auc:
        print(f"  ROC-AUC:      {roc_auc:.4f} ({roc_auc*100:.2f}%)")

    print(f"\n{'─'*80}")
    print("VALIDACIÓN CRUZADA (5-FOLD):")
    print('─'*80)
    print(f"  Media:        {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
    print(f"  Rango:        [{cv_scores.min():.4f}, {cv_scores.max():.4f}]")

    print(f"\n{'─'*80}")
    print("MATRIZ DE CONFUSIÓN:")
    print('─'*80)
    print(f"\n{cm}")
    print(f"\n  VN: {cm[0,0]:>6}  |  FP: {cm[0,1]:>6}")
    print(f"  FN: {cm[1,0]:>6}  |  VP: {cm[1,1]:>6}")

    # =========================================================================
    # GENERAR GRÁFICOS ESPECÍFICOS PARA ESTE MODELO
    # =========================================================================
    print(f"\n{'─'*80}")
    print("GENERANDO GRÁFICOS...")
    print('─'*80)

    # Limpiar nombre para archivo
    nombre_archivo = nombre_modelo.replace(' ', '_').replace('/', '_')

    # FIGURA COMPLETA: 2x2 grid
    fig = plt.figure(figsize=(16, 12))
    fig.suptitle(f'{nombre_modelo} - Análisis Completo', fontsize=16, fontweight='bold')

    # ----------- SUBPLOT 1: Matriz de Confusión -----------
    ax1 = plt.subplot(2, 2, 1)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['No cepilla', 'Sí cepilla'],
                yticklabels=['No cepilla', 'Sí cepilla'],
                ax=ax1, annot_kws={'size': 14, 'weight': 'bold'})
    ax1.set_xlabel('Predicción', fontsize=11, fontweight='bold')
    ax1.set_ylabel('Valor Real', fontsize=11, fontweight='bold')
    ax1.set_title('Matriz de Confusión', fontsize=12, fontweight='bold')

    # Agregar texto con interpretación
    total = cm.sum()
    correctos = cm[0,0] + cm[1,1]
    ax1.text(0.5, -0.25, f'Accuracy: {correctos}/{total} ({accuracy*100:.1f}%)',
             transform=ax1.transAxes, ha='center', fontsize=10,
             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))

    # ----------- SUBPLOT 2: Barras de Métricas -----------
    ax2 = plt.subplot(2, 2, 2)
    metricas_valores = [accuracy, precision, recall, f1]
    metricas_nombres = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    colores = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']

    barras = ax2.bar(metricas_nombres, metricas_valores, color=colores, alpha=0.7, edgecolor='black')
    ax2.set_ylim([0, 1])
    ax2.set_ylabel('Valor', fontsize=11, fontweight='bold')
    ax2.set_title('Métricas de Evaluación', fontsize=12, fontweight='bold')
    ax2.grid(axis='y', alpha=0.3)

    # Añadir valores sobre las barras
    for i, (bar, val) in enumerate(zip(barras, metricas_valores)):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{val:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

    # ----------- SUBPLOT 3: Curva ROC -----------
    ax3 = plt.subplot(2, 2, 3)
    if tiene_proba:
        fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
        ax3.plot(fpr, tpr, linewidth=2.5, label=f'ROC (AUC={roc_auc:.3f})', color='#e74c3c')
        ax3.plot([0, 1], [0, 1], 'k--', linewidth=1.5, label='Aleatorio (AUC=0.500)')
        ax3.fill_between(fpr, tpr, alpha=0.2, color='#e74c3c')
        ax3.set_xlabel('Tasa de Falsos Positivos (FPR)', fontsize=11, fontweight='bold')
        ax3.set_ylabel('Tasa de Verdaderos Positivos (TPR)', fontsize=11, fontweight='bold')
        ax3.set_title('Curva ROC', fontsize=12, fontweight='bold')
        ax3.legend(loc='lower right', fontsize=10)
        ax3.grid(True, alpha=0.3)

        # Marcar punto óptimo
        optimal_idx = np.argmax(tpr - fpr)
        optimal_threshold = thresholds[optimal_idx]
        ax3.plot(fpr[optimal_idx], tpr[optimal_idx], 'go', markersize=10,
                label=f'Óptimo (umbral={optimal_threshold:.2f})')
        ax3.legend(loc='lower right', fontsize=9)
    else:
        ax3.text(0.5, 0.5, 'No disponible\n(modelo sin probabilidades)',
                ha='center', va='center', fontsize=12, transform=ax3.transAxes)
        ax3.set_title('Curva ROC', fontsize=12, fontweight='bold')

    # ----------- SUBPLOT 4: Validación Cruzada -----------
    ax4 = plt.subplot(2, 2, 4)

    # Boxplot de CV scores
    bp = ax4.boxplot([cv_scores], widths=0.5, patch_artist=True,
                     boxprops=dict(facecolor='lightgreen', alpha=0.7),
                     medianprops=dict(color='red', linewidth=2),
                     whiskerprops=dict(linewidth=1.5),
                     capprops=dict(linewidth=1.5))

    # Scatter plot de los scores individuales
    ax4.scatter([1]*len(cv_scores), cv_scores, color='darkgreen', s=100,
               zorder=3, alpha=0.6, edgecolors='black')

    ax4.set_ylabel('Accuracy', fontsize=11, fontweight='bold')
    ax4.set_title('Validación Cruzada (5-Fold)', fontsize=12, fontweight='bold')
    ax4.set_xticklabels(['CV Scores'])
    ax4.grid(axis='y', alpha=0.3)
    ax4.set_ylim([max(0, cv_scores.min() - 0.05), min(1, cv_scores.max() + 0.05)])

    # Añadir línea de media
    ax4.axhline(y=cv_scores.mean(), color='blue', linestyle='--', linewidth=2,
               label=f'Media: {cv_scores.mean():.3f}±{cv_scores.std():.3f}')
    ax4.legend(loc='lower right', fontsize=9)

    # Añadir texto con scores individuales
    scores_text = 'Scores: ' + ', '.join([f'{s:.3f}' for s in cv_scores])
    ax4.text(0.5, -0.15, scores_text, transform=ax4.transAxes,
            ha='center', fontsize=9, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    plt.tight_layout()
    nombre_grafico = f'Modelo_{numero_modelo}_{nombre_archivo}.png'
    plt.savefig(nombre_grafico, dpi=300, bbox_inches='tight')
    print(f"  ✓ Guardado: {nombre_grafico}")
    plt.close()

    # =========================================================================
    # INTERPRETACIÓN AUTOMÁTICA
    # =========================================================================
    print(f"\n{'─'*80}")
    print("INTERPRETACIÓN:")
    print('─'*80)

    # Interpretación de Accuracy
    if accuracy >= 0.90:
        print(f"  • Accuracy EXCELENTE ({accuracy*100:.1f}%): El modelo es altamente confiable.")
    elif accuracy >= 0.80:
        print(f"  • Accuracy BUENA ({accuracy*100:.1f}%): El modelo tiene buen desempeño general.")
    elif accuracy >= 0.70:
        print(f"  • Accuracy ACEPTABLE ({accuracy*100:.1f}%): El modelo funciona razonablemente.")
    else:
        print(f"  • Accuracy BAJA ({accuracy*100:.1f}%): El modelo necesita mejoras.")

    # Interpretación de Precision vs Recall
    if precision > recall + 0.05:
        print(f"  • ALTA PRECISION ({precision*100:.1f}%), RECALL MODERADO ({recall*100:.1f}%):")
        print(f"    El modelo es conservador - pocas falsas alarmas pero pierde algunos casos.")
    elif recall > precision + 0.05:
        print(f"  • ALTO RECALL ({recall*100:.1f}%), PRECISION MODERADA ({precision*100:.1f}%):")
        print(f"    El modelo es sensible - detecta la mayoría de casos pero con más falsas alarmas.")
    else:
        print(f"  • PRECISION ({precision*100:.1f}%) y RECALL ({recall*100:.1f}%) BALANCEADOS:")
        print(f"    El modelo tiene equilibrio entre detectar casos y evitar falsas alarmas.")

    # Interpretación de F1-Score
    if f1 >= 0.85:
        print(f"  • F1-Score EXCELENTE ({f1*100:.1f}%): Balance óptimo precision-recall.")
    elif f1 >= 0.75:
        print(f"  • F1-Score BUENO ({f1*100:.1f}%): Buen balance general.")
    else:
        print(f"  • F1-Score MEJORABLE ({f1*100:.1f}%): Revisar umbral de decisión.")

    # Interpretación de Validación Cruzada
    cv_std = cv_scores.std()
    if cv_std < 0.02:
        print(f"  • ALTA ESTABILIDAD (σ={cv_std:.4f}): Modelo muy robusto y consistente.")
    elif cv_std < 0.05:
        print(f"  • BUENA ESTABILIDAD (σ={cv_std:.4f}): Modelo confiable.")
    else:
        print(f"  • ESTABILIDAD VARIABLE (σ={cv_std:.4f}): Desempeño depende de los datos.")

    # Interpretación de ROC-AUC
    if roc_auc:
        if roc_auc >= 0.90:
            print(f"  • ROC-AUC EXCELENTE ({roc_auc*100:.1f}%): Capacidad discriminativa superior.")
        elif roc_auc >= 0.80:
            print(f"  • ROC-AUC BUENA ({roc_auc*100:.1f}%): Buena capacidad de separación de clases.")
        elif roc_auc >= 0.70:
            print(f"  • ROC-AUC ACEPTABLE ({roc_auc*100:.1f}%): Capacidad discriminativa moderada.")
        else:
            print(f"  • ROC-AUC BAJA ({roc_auc*100:.1f}%): Limitada capacidad discriminativa.")

    # Análisis de errores
    fn = cm[1, 0]  # Falsos Negativos
    fp = cm[0, 1]  # Falsos Positivos
    total_errors = fn + fp

    if total_errors > 0:
        print(f"\n  • ANÁLISIS DE ERRORES:")
        print(f"    - Falsos Negativos: {fn} ({fn/total_errors*100:.1f}% de errores)")
        print(f"      → Niños que SÍ se cepillan pero predichos como NO")
        print(f"    - Falsos Positivos: {fp} ({fp/total_errors*100:.1f}% de errores)")
        print(f"      → Niños que NO se cepillan pero predichos como SÍ")

        if fn > fp * 1.5:
            print(f"    ⚠ PREDOMINAN FALSOS NEGATIVOS: Ajustar umbral para detectar más casos.")
        elif fp > fn * 1.5:
            print(f"    ⚠ PREDOMINAN FALSOS POSITIVOS: Ajustar umbral para ser más selectivo.")

    return {
        'Modelo': nombre_modelo,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': roc_auc,
        'CV_Mean': cv_scores.mean(),
        'CV_Std': cv_scores.std(),
        'CM': cm,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'cv_scores': cv_scores
    }


In [12]:
# =============================================================================
# PASO 7: ENTRENAMIENTO Y EVALUACIÓN DE MODELOS
# =============================================================================
print("\n" + "="*80)
print("PASO 7: ENTRENAMIENTO DE LOS 7 MODELOS")
print("="*80)

resultados_todos = []
modelos_entrenados = {}


PASO 7: ENTRENAMIENTO DE LOS 7 MODELOS


In [13]:
# -------------------------------------------------------------------------
# MODELO 1: REGRESIÓN LOGÍSTICA
# -------------------------------------------------------------------------
print("\n" + "─"*80)
print("MODELO 1/7: REGRESIÓN LOGÍSTICA")
print("─"*80)

lr = LogisticRegression(max_iter=1000, random_state=42, solver='lbfgs')
lr.fit(X_train_scaled, y_train)

resultados_lr = evaluar_modelo('Regresión Logística', lr, X_train_scaled, X_test_scaled, y_train, y_test, numero_modelo=1)
resultados_todos.append(resultados_lr)
modelos_entrenados['Logistic Regression'] = lr


────────────────────────────────────────────────────────────────────────────────
MODELO 1/7: REGRESIÓN LOGÍSTICA
────────────────────────────────────────────────────────────────────────────────

EVALUANDO: Regresión Logística

────────────────────────────────────────────────────────────────────────────────
MÉTRICAS EN CONJUNTO DE PRUEBA:
────────────────────────────────────────────────────────────────────────────────
  Accuracy:     0.8038 (80.38%)
  Precision:    0.8045 (80.45%)
  Recall:       0.9984 (99.84%)
  F1-Score:     0.8910 (89.10%)
  ROC-AUC:      0.6605 (66.05%)

────────────────────────────────────────────────────────────────────────────────
VALIDACIÓN CRUZADA (5-FOLD):
────────────────────────────────────────────────────────────────────────────────
  Media:        0.8024 ± 0.0005
  Rango:        [0.8015, 0.8031]

────────────────────────────────────────────────────────────────────────────────
MATRIZ DE CONFUSIÓN:
──────────────────────────────────────────────────────────

In [14]:
# -------------------------------------------------------------------------
# MODELO 2: SUPPORT VECTOR CLASSIFIER
# -------------------------------------------------------------------------
print("\n" + "─"*80)
print("MODELO 2/7: SUPPORT VECTOR CLASSIFIER (SVC)")
print("─"*80)

svc = SVC(kernel='rbf', C=1.0, gamma='scale', probability=True, random_state=42)
svc.fit(X_train_scaled, y_train)

resultados_svc = evaluar_modelo('SVC', svc, X_train_scaled, X_test_scaled, y_train, y_test, numero_modelo=2)
resultados_todos.append(resultados_svc)
modelos_entrenados['SVC'] = svc


────────────────────────────────────────────────────────────────────────────────
MODELO 2/7: SUPPORT VECTOR CLASSIFIER (SVC)
────────────────────────────────────────────────────────────────────────────────

EVALUANDO: SVC

────────────────────────────────────────────────────────────────────────────────
MÉTRICAS EN CONJUNTO DE PRUEBA:
────────────────────────────────────────────────────────────────────────────────
  Accuracy:     0.8120 (81.20%)
  Precision:    0.8229 (82.29%)
  Recall:       0.9760 (97.60%)
  F1-Score:     0.8930 (89.30%)
  ROC-AUC:      0.8231 (82.31%)

────────────────────────────────────────────────────────────────────────────────
VALIDACIÓN CRUZADA (5-FOLD):
────────────────────────────────────────────────────────────────────────────────
  Media:        0.8104 ± 0.0019
  Rango:        [0.8071, 0.8127]

────────────────────────────────────────────────────────────────────────────────
MATRIZ DE CONFUSIÓN:
──────────────────────────────────────────────────────────────

In [15]:
# -------------------------------------------------------------------------
# MODELO 3: GAUSSIAN NAIVE BAYES
# -------------------------------------------------------------------------
print("\n" + "─"*80)
print("MODELO 3/7: GAUSSIAN NAIVE BAYES")
print("─"*80)

gnb = GaussianNB()
gnb.fit(X_train_scaled, y_train)

resultados_gnb = evaluar_modelo('Gaussian Naive Bayes', gnb, X_train_scaled, X_test_scaled, y_train, y_test, numero_modelo=3)
resultados_todos.append(resultados_gnb)
modelos_entrenados['Gaussian NB'] = gnb


────────────────────────────────────────────────────────────────────────────────
MODELO 3/7: GAUSSIAN NAIVE BAYES
────────────────────────────────────────────────────────────────────────────────

EVALUANDO: Gaussian Naive Bayes

────────────────────────────────────────────────────────────────────────────────
MÉTRICAS EN CONJUNTO DE PRUEBA:
────────────────────────────────────────────────────────────────────────────────
  Accuracy:     0.6551 (65.51%)
  Precision:    0.9931 (99.31%)
  Recall:       0.5747 (57.47%)
  F1-Score:     0.7281 (72.81%)
  ROC-AUC:      0.8331 (83.31%)

────────────────────────────────────────────────────────────────────────────────
VALIDACIÓN CRUZADA (5-FOLD):
────────────────────────────────────────────────────────────────────────────────
  Media:        0.6584 ± 0.0011
  Rango:        [0.6568, 0.6600]

────────────────────────────────────────────────────────────────────────────────
MATRIZ DE CONFUSIÓN:
────────────────────────────────────────────────────────

In [16]:
# -------------------------------------------------------------------------
# MODELO 4: K-NEIGHBORS CLASSIFIER
# -------------------------------------------------------------------------
print("\n" + "─"*80)
print("MODELO 4/7: K-NEIGHBORS CLASSIFIER")
print("─"*80)

knn = KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='euclidean')
knn.fit(X_train_scaled, y_train)

resultados_knn = evaluar_modelo('K-Neighbors', knn, X_train_scaled, X_test_scaled, y_train, y_test, numero_modelo=4)
resultados_todos.append(resultados_knn)
modelos_entrenados['KNeighbors'] = knn


────────────────────────────────────────────────────────────────────────────────
MODELO 4/7: K-NEIGHBORS CLASSIFIER
────────────────────────────────────────────────────────────────────────────────

EVALUANDO: K-Neighbors

────────────────────────────────────────────────────────────────────────────────
MÉTRICAS EN CONJUNTO DE PRUEBA:
────────────────────────────────────────────────────────────────────────────────
  Accuracy:     0.7879 (78.79%)
  Precision:    0.8491 (84.91%)
  Recall:       0.8951 (89.51%)
  F1-Score:     0.8715 (87.15%)
  ROC-AUC:      0.7781 (77.81%)

────────────────────────────────────────────────────────────────────────────────
VALIDACIÓN CRUZADA (5-FOLD):
────────────────────────────────────────────────────────────────────────────────
  Media:        0.7896 ± 0.0054
  Rango:        [0.7830, 0.7979]

────────────────────────────────────────────────────────────────────────────────
MATRIZ DE CONFUSIÓN:
───────────────────────────────────────────────────────────────

In [17]:
# -------------------------------------------------------------------------
# MODELO 5: RANDOM FOREST
# -------------------------------------------------------------------------
print("\n" + "─"*80)
print("MODELO 5/7: RANDOM FOREST")
print("─"*80)

rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train_scaled, y_train)

resultados_rf = evaluar_modelo('Random Forest', rf, X_train_scaled, X_test_scaled, y_train, y_test, numero_modelo=5)
resultados_todos.append(resultados_rf)
modelos_entrenados['Random Forest'] = rf

print(f"\nImportancia de características (Top 5):")
importancia = pd.DataFrame({
    'Variable': X.columns,
    'Importancia': rf.feature_importances_
}).sort_values('Importancia', ascending=False).head()
print(importancia.to_string(index=False))


────────────────────────────────────────────────────────────────────────────────
MODELO 5/7: RANDOM FOREST
────────────────────────────────────────────────────────────────────────────────

EVALUANDO: Random Forest

────────────────────────────────────────────────────────────────────────────────
MÉTRICAS EN CONJUNTO DE PRUEBA:
────────────────────────────────────────────────────────────────────────────────
  Accuracy:     0.8076 (80.76%)
  Precision:    0.8332 (83.32%)
  Recall:       0.9510 (95.10%)
  F1-Score:     0.8882 (88.82%)
  ROC-AUC:      0.8405 (84.05%)

────────────────────────────────────────────────────────────────────────────────
VALIDACIÓN CRUZADA (5-FOLD):
────────────────────────────────────────────────────────────────────────────────
  Media:        0.8114 ± 0.0021
  Rango:        [0.8091, 0.8147]

────────────────────────────────────────────────────────────────────────────────
MATRIZ DE CONFUSIÓN:
──────────────────────────────────────────────────────────────────────

In [18]:
# -------------------------------------------------------------------------
# MODELO 6: ADABOOST
# -------------------------------------------------------------------------
print("\n" + "─"*80)
print("MODELO 6/7: ADABOOST CLASSIFIER")
print("─"*80)

ada = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1, random_state=42),
    n_estimators=50,
    learning_rate=1.0,
    random_state=42,
    algorithm='SAMME'
)
ada.fit(X_train_scaled, y_train)

resultados_ada = evaluar_modelo('AdaBoost', ada, X_train_scaled, X_test_scaled, y_train, y_test, numero_modelo=6)
resultados_todos.append(resultados_ada)
modelos_entrenados['AdaBoost'] = ada


────────────────────────────────────────────────────────────────────────────────
MODELO 6/7: ADABOOST CLASSIFIER
────────────────────────────────────────────────────────────────────────────────

EVALUANDO: AdaBoost

────────────────────────────────────────────────────────────────────────────────
MÉTRICAS EN CONJUNTO DE PRUEBA:
────────────────────────────────────────────────────────────────────────────────
  Accuracy:     0.8035 (80.35%)
  Precision:    0.8035 (80.35%)
  Recall:       1.0000 (100.00%)
  F1-Score:     0.8910 (89.10%)
  ROC-AUC:      0.8124 (81.24%)

────────────────────────────────────────────────────────────────────────────────
VALIDACIÓN CRUZADA (5-FOLD):
────────────────────────────────────────────────────────────────────────────────
  Media:        0.8020 ± 0.0029
  Rango:        [0.7963, 0.8035]

────────────────────────────────────────────────────────────────────────────────
MATRIZ DE CONFUSIÓN:
────────────────────────────────────────────────────────────────────

In [19]:
# -------------------------------------------------------------------------
# MODELO 7: GRADIENT BOOSTING
# -------------------------------------------------------------------------
print("\n" + "─"*80)
print("MODELO 7/7: GRADIENT BOOSTING CLASSIFIER")
print("─"*80)

gb = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    min_samples_split=5,
    min_samples_leaf=2,
    subsample=0.8,
    random_state=42
)
gb.fit(X_train_scaled, y_train)

resultados_gb = evaluar_modelo('Gradient Boosting', gb, X_train_scaled, X_test_scaled, y_train, y_test, numero_modelo=7)
resultados_todos.append(resultados_gb)
modelos_entrenados['Gradient Boosting'] = gb

print(f"\nImportancia de características (Top 5):")
importancia_gb = pd.DataFrame({
    'Variable': X.columns,
    'Importancia': gb.feature_importances_
}).sort_values('Importancia', ascending=False).head()
print(importancia_gb.to_string(index=False))



────────────────────────────────────────────────────────────────────────────────
MODELO 7/7: GRADIENT BOOSTING CLASSIFIER
────────────────────────────────────────────────────────────────────────────────

EVALUANDO: Gradient Boosting

────────────────────────────────────────────────────────────────────────────────
MÉTRICAS EN CONJUNTO DE PRUEBA:
────────────────────────────────────────────────────────────────────────────────
  Accuracy:     0.8115 (81.15%)
  Precision:    0.8328 (83.28%)
  Recall:       0.9576 (95.76%)
  F1-Score:     0.8909 (89.09%)
  ROC-AUC:      0.8467 (84.67%)

────────────────────────────────────────────────────────────────────────────────
VALIDACIÓN CRUZADA (5-FOLD):
────────────────────────────────────────────────────────────────────────────────
  Media:        0.8141 ± 0.0031
  Rango:        [0.8107, 0.8193]

────────────────────────────────────────────────────────────────────────────────
MATRIZ DE CONFUSIÓN:
───────────────────────────────────────────────────

In [20]:
# =============================================================================
# RESUMEN COMPARATIVO
# =============================================================================
print("\n" + "="*80)
print("RESUMEN COMPARATIVO DE TODOS LOS MODELOS")
print("="*80)

df_resultados = pd.DataFrame(resultados_todos)
df_resultados_sorted = df_resultados.sort_values('F1-Score', ascending=False)

print("\n" + "─"*80)
print("TABLA COMPARATIVA")
print("─"*80)
print(df_resultados_sorted[['Modelo', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC', 'CV_Mean']].to_string(index=False))

mejor = df_resultados_sorted.iloc[0]
print(f"\n{'─'*80}")
print(f"🏆 MEJOR MODELO: {mejor['Modelo']}")
print(f"   F1-Score: {mejor['F1-Score']:.4f} ({mejor['F1-Score']*100:.2f}%)")
print("─"*80)


RESUMEN COMPARATIVO DE TODOS LOS MODELOS

────────────────────────────────────────────────────────────────────────────────
TABLA COMPARATIVA
────────────────────────────────────────────────────────────────────────────────
              Modelo  Accuracy  Precision   Recall  F1-Score  ROC-AUC  CV_Mean
                 SVC  0.811978   0.822915 0.976019  0.892952 0.823081 0.810397
 Regresión Logística  0.803789   0.804509 0.998401  0.891029 0.660451 0.802369
            AdaBoost  0.803468   0.803468 1.000000  0.891026 0.812389 0.802047
   Gradient Boosting  0.811496   0.832812 0.957634  0.890872 0.846748 0.814091
       Random Forest  0.807643   0.833158 0.951039  0.888205 0.840466 0.811441
         K-Neighbors  0.787893   0.849100 0.895084  0.871486 0.778122 0.789562
Gaussian Naive Bayes  0.655106   0.993094 0.574740  0.728101 0.833110 0.658370

────────────────────────────────────────────────────────────────────────────────
🏆 MEJOR MODELO: SVC
   F1-Score: 0.8930 (89.30%)
──────────────

In [21]:
# =============================================================================
# VISUALIZACIONES
# =============================================================================
print("\n" + "="*80)
print("GENERANDO VISUALIZACIONES")
print("="*80)

# 1. Comparación de métricas
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Comparación de Métricas por Modelo', fontsize=16, fontweight='bold')

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']

for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    data_sorted = df_resultados.sort_values(metric, ascending=True)

    bars = ax.barh(data_sorted['Modelo'], data_sorted[metric], color=colors[idx], alpha=0.7)
    ax.set_xlabel(metric, fontsize=12, fontweight='bold')
    ax.set_title(f'{metric}', fontsize=12, fontweight='bold')
    ax.set_xlim([0, 1])
    ax.grid(axis='x', alpha=0.3)

    for i, (v, bar) in enumerate(zip(data_sorted[metric], bars)):
        ax.text(v + 0.01, i, f'{v:.3f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig('01_comparacion_metricas.png', dpi=300, bbox_inches='tight')
print("✓ Guardado: 01_comparacion_metricas.png")
plt.close()

# 2. Curvas ROC
fig, ax = plt.subplots(figsize=(10, 8))

for resultado in resultados_todos:
    if resultado['y_pred_proba'] is not None:
        fpr, tpr, _ = roc_curve(y_test, resultado['y_pred_proba'])
        auc = resultado['ROC-AUC']
        ax.plot(fpr, tpr, linewidth=2, label=f"{resultado['Modelo']} (AUC={auc:.3f})")

ax.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Aleatorio (AUC=0.500)')
ax.set_xlabel('Tasa de Falsos Positivos', fontsize=12, fontweight='bold')
ax.set_ylabel('Tasa de Verdaderos Positivos', fontsize=12, fontweight='bold')
ax.set_title('Curvas ROC', fontsize=14, fontweight='bold')
ax.legend(loc='lower right', fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('02_curvas_roc.png', dpi=300, bbox_inches='tight')
print("✓ Guardado: 02_curvas_roc.png")
plt.close()


GENERANDO VISUALIZACIONES
✓ Guardado: 01_comparacion_metricas.png
✓ Guardado: 02_curvas_roc.png
